# Regression of Vote Characteristics on CAR

In [52]:
import stata_setup

stata_setup.config('/Applications/StataNow', 'mp')

In [53]:
%%stata

clear all
set more off
set varabbrev off
version 18

* Paths
global PROCESSED_DATA "processed_data"
global TABLES "tables"


. 
. clear all

. set more off

. set varabbrev off

. version 18

. 
. * Paths
. global PROCESSED_DATA "processed_data"

. global TABLES "tables"

. 


In [ ]:
%%stata
foreach stage in created end {
    import delimited using "$PROCESSED_DATA/event_study_panel_`stage'.csv", clear

    * Parse date
    gen day = date(date, "YMD")
    format day %td

    * Create year variable
    gen year = year(day)

    * Create quarter variable
    gen quarter = qofd(day)
    format quarter %tq

    * Ensure index is numeric (safeguard if it came in as string)
    capture confirm numeric variable index
    if _rc {
        destring index, replace ignore("[]")
    }

    * Encode categorical variables
    encode gecko_id, gen(token)
    encode space,    gen(dao)

    egen token_year = group(token year)
    egen token_quarter = group(token quarter)

    * Define windows and specs
    local windows 2 5
    local vars    half_vp_ratio vn_hhi vs_hhi cci

    eststo clear

    foreach w of local windows {
        preserve
        keep if index == `w'

        foreach v of local vars {

            * Run regression
            reghdfe car_`w' `v', absorb(token year) vce(cluster token)
            eststo car`w'_`v'

            * Tag FE rows (Y/N) for this spec
            estadd local fe_token "Y"
            estadd local fe_time  "Y"
        }

        restore
    }

    * Export a single big LaTeX table with 8 models (2 windows × 4 specs)
    esttab                                                         ///
        car2_half_vp_ratio car2_vn_hhi car2_vs_hhi car2_cci        ///
        car5_half_vp_ratio car5_vn_hhi car5_vs_hhi car5_cci        ///
        using "$TABLES/reg_car_votes_`stage'.tex", replace         ///
        se star(* 0.10 ** 0.05 *** 0.01) label nogaps nocon        ///
        nonotes booktabs nomtitle                                  ///
        mgroups("CAR[-2,2]" "CAR[-5,5]",                           ///
            span                                                   ///
            pattern(1 0 0 0 1 0 0 0)                               ///
            prefix(\multicolumn{@span}{c}{)                        ///
            suffix(}))                                             ///
        stats(fe_token fe_time N r2_a,                             ///      
              fmt(0 0 %9.0fc %9.3f) labels("Token FE" "Year FE" "Observations" "Adjusted R²"))
}


. foreach stage in created end {
  2.     import delimited using "$PROCESSED_DATA/event_study_panel_`stage'.csv"
> , clear
  3. 
.     * Parse date
.     gen day = date(date, "YMD")
  4.     format day %td
  5. 
.     * Create year variable
.     gen year = year(day)
  6. 
.     * Create quarter variable
.     gen quarter = qofd(day)
  7.     format quarter %tq
  8. 
.     * Ensure index is numeric (safeguard if it came in as string)
.     capture confirm numeric variable index
  9.     if _rc {
 10.         destring index, replace ignore("[]")
 11.     }
 12. 
.     * Encode categorical variables
.     encode gecko_id, gen(token)
 13.     encode space,    gen(dao)
 14. 
.     egen token_year = group(token year)
 15.     egen token_quarter = group(token quarter)
 16. 
.     * Define windows and specs
.     local windows 2 5
 17.     local vars    half_vp_ratio vn_hhi vs_hhi cci
 18. 
.     eststo clear
 19. 
.     foreach w of local windows {
 20.         preserve
 21.         keep if